In [1]:
# import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import RK45
from functools import partial
from scipy.interpolate import CubicSpline
# from scipy.integrate import solve_ivp
# import matplotlib.pyplot as plt
# import pandas as pd
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': 20})
rc('text', usetex=True)

# checks s foe far field


# event_s_check_far.terminal = True  # Stop the integration when this event occurs
# event_s_check_far.direction = -1  # Detect zero crossing in any direction

# dim = bb1, bb2, bb3, dd1 ,dd2, dd3, dd4, ee1, ee2, ll1, ll2, ll3, mm0, mm1, mm2

def s_far(s, a1, a2):
    # Xoxxx, Xozzz, Xdxxx, Xdzzz = state
    # r = np.sqrt(x ** 2 + z ** 2)
    # p,a,dim = args
    # a2 = a
    # a1, rho, Fz, Fx = p
    la = a2 / a1
    # x = Xoxxx - Xdxxx
    # z = Xozzz - Xdzzz
    # s = (2 * r) / (a2 + a1)
    # if s < 2.0001: s = 2.0001
    term1B11 = (68 * la ** 5) / ((1 + la) ** 6 * s ** 6)
    term2B11 = (32 * la ** 3 * (10 - 9 * la ** 2 + 9 * la ** 4)) / ((1 + la) ** 8 * s ** 8)
    term3B11 = (192 * la ** 5 * (35 - 18 * la ** 2 + 6 * la ** 4)) / ((1 + la) ** 10 * s ** 10)

    B11 = 1 - (term1B11 + term2B11 + term3B11)

    term1A11 = (60 * la ** 3) / ((1 + la) ** 4 * s ** 4)
    term2A11 = (60 * la ** 3 * (8 - la ** 2)) / ((1 + la) ** 6 * s ** 6)
    term3A11 = (32 * la ** 3 * (20 - 123 * la ** 2 + 9 * la ** 4)) / ((1 + la) ** 8 * s ** 8)
    term4A11 = (64 * la ** 2 * (175 + 1500 * la - 426 * la ** 2 + 18 * la ** 4)) / ((1 + la) ** 10 * s ** 10)
    A11 = B11 - (term1A11 - term2A11 + term3A11 + term4A11)

    term1L = 1
    term2L = -3 / ((1 + la) * s)
    term3L = 4 * (1 + la ** 2) / ((1 + la) ** 3 * s ** 3)
    term4L = -60 * la ** 3 / ((1 + la) ** 4 * s ** 4)
    term5L = 32 * la ** 3 * (15 - 4 * la ** 2) / ((1 + la) ** 6 * s ** 6)
    term6L = -2400 * la ** 3 / ((1 + la) ** 7 * s ** 7)
    term7L = -192 * la ** 3 * (5 - 22 * la ** 2 + 3 * la ** 4) / ((1 + la) ** 8 * s ** 8)
    term8L = 1920 * la ** 3 * (1 + la ** 2) / ((1 + la) ** 9 * s ** 9)
    term9L = -256 * la ** 5 * (70 - 375 * la - 120 * la ** 2 + 9 * la ** 3) / ((1 + la) ** 10 * s ** 10)
    term10L = -1536 * la ** 3 * (10 - 151 * la ** 2 + 10 * la ** 4) / ((1 + la) ** 11 * s ** 11)
    L = term1L + term2L + term3L + term4L + term5L + term6L + term7L + term8L + term9L + term10L

    term1M = 1
    term2M = -3 / (2 * (1 + la) * s)
    term3M = -2 * (1 + la ** 2) / ((1 + la) ** 3 * s ** 3)
    term4M = -68 * la ** 5 / ((1 + la) ** 6 * s ** 6)
    term5M = -32 * la ** 3 * (10 - 9 * la ** 2 + 9 * la ** 4) / ((1 + la) ** 8 * s ** 8)
    term6M = -192 * la ** 5 * (35 - 18 * la ** 2 + 6 * la ** 4) / ((1 + la) ** 10 * s ** 10)
    term7M = -16 * la ** 3 * (560 - 553 * la ** 2 + 560 * la ** 4) / ((1 + la) ** 11 * s ** 11)
    M = term1M + term2M + term3M + term4M + term5M + term6M + term7M

    A12 = (A11 - L) * ((1 + la) / 2)
    B12 = (B11 - M) * ((1 + la) / 2)
    # dXodt = (Fz / (3 * np.pi * rho * (a1 + a2))) * (((A12 - B12) / r ** 2) * (Xoz - Xdz) * (Xox - Xdx))
    # dZodt = (Fz / (3 * np.pi * rho * (a1 + a2))) * ((((A12 - B12) / r ** 2) * ((Xoz - Xdz) ** 2)) + B12)
    # dXddt = (Fz / (6 * np.pi * rho * a1)) * (((A11 - B11) / r ** 2) * (Xoz - Xdz) * (Xox - Xdx))
    # # dZddt = (Fz / (6 * np.pi * rho * a1)) * ((((A11 - B11) / r ** 2) * ((Xoz - Xdz) ** 2)) + B11)
    # dx_ddt = (1 / (6 * np.pi * rho * a1)) * (
    #         A11 * (Fx * x + Fz * z) * x / r ** 2 + B11 * Fx - B11 * (Fx * x + Fz * z) * x / r ** 2)
    # dz_ddt = (1 / (6 * np.pi * rho * a1)) * (
    #         A11 * (Fx * x + Fz * z) * z / r ** 2 + B11 * Fz - B11 * (Fx * x + Fz * z) * z / r ** 2)
    # dx_odt = (1 / (3 * np.pi * rho * (a1 + a2))) * (
    #         A12 * (Fx * x + Fz * z) * x / r ** 2 + B12 * Fx - B12 * (Fx * x + Fz * z) * x / r ** 2)
    # dz_odt = (1 / (3 * np.pi * rho * (a1 + a2))) * (
    #         A12 * (Fx * x + Fz * z) * z / r ** 2 + B12 * Fz - B12 * (Fx * x + Fz * z) * z / r ** 2)
    # return [dx_odt, dz_odt, dx_ddt, dz_ddt]
    return A11, B11, L, M, A12, B11


def s_near(s,a1,a2,dim): 
    # Xoxx, Xozz, Xdxx, Xdzz= state
    # r = np.sqrt(x ** 2 + z ** 2)
    # p, a, dim = args
    # a2 = a
    # a1, rho, Fz, Fx = p
    la = a2 / a1
    # x = Xoxx - Xdxx
    # z = Xozz - Xdzz
    # s = (2 * r) / (a2 + a1)
    # if s < 2.0001: s = 2.000101
    # print(state)
    # Define coefficients
    # d1, d2, d3, d4 = 0.7750, 0.9306, -0.900, -2.0
    # b1, b2, b3, e1, e2 = 0.8905, 5.772, 7.070, 6.043, 6.327
    # l1, l2, l3 = 2.0, -1.80, -4.0
    # m0, m1, m2 = 0.4021, 2.967, 5.088
    b1, b2, b3, d1, d2, d3, d4, e1, e2, l1, l2, l3, m0, m1, m2 = dim
    xi_value = s - 2
    # if xi_value <= 0:
    #     print(xi_value)
    #     xi_value = 0.01
    # Calculate the A_11 value with the error term
    A11 = d1 + d2 * xi_value + d3 * xi_value ** 2 * np.log(1 / xi_value) + d4 * xi_value ** 2

    # Calculate the B_11 value
    B11 = (b1 * (np.log(1 / xi_value)) ** 2 + b2 * np.log(1 / xi_value) + b3) / \
          ((np.log(1 / xi_value)) ** 2 + e1 * np.log(1 / xi_value) + e2)

    # Calculate the L value
    L = l1 * xi_value + l2 * xi_value ** 2 * np.log(1 / xi_value) + l3 * xi_value ** 2

    # Calculate the M value
    M = (m0 * (np.log(1 / xi_value)) ** 2 + m1 * np.log(1 / xi_value) + m2) / \
        ((np.log(1 / xi_value)) ** 2 + e1 * np.log(1 / xi_value) + e2)

    # if s - 2 <= 0: print(f's - 2 = {s - 2}\nA11 = {A11}\nB11 = {B11}\nL = {L}\nM ={M}')

    A12 = (A11 - L) * ((1 + la) / 2)
    B12 = (B11 - M) * ((1 + la) / 2)
    # dx_ddt = (1 / (6 * np.pi * rho * a1)) * (
    #         A11 * (Fx * x + Fz * z) * x / r ** 2 + B11 * Fx - B11 * (Fx * x + Fz * z) * x / r ** 2)
    # dz_ddt = (1 / (6 * np.pi * rho * a1)) * (
    #         A11 * (Fx * x + Fz * z) * z / r ** 2 + B11 * Fz - B11 * (Fx * x + Fz * z) * z / r ** 2)
    # dx_odt = (1 / (3 * np.pi * rho * (a1 + a2))) * (
    #         A12 * (Fx * x + Fz * z) * x / r ** 2 + B12 * Fx - B12 * (Fx * x + Fz * z) * x / r ** 2)
    # dz_odt = (1 / (3 * np.pi * rho * (a1 + a2))) * (
    #         A12 * (Fx * x + Fz * z) * z / r ** 2 + B12 * Fz - B12 * (Fx * x + Fz * z) * z / r ** 2)
    # # if s - 2 <= 0: print(f's - 2 = {s - 2}\nA11 = {A11}\nB11 = {B11}\nA12 = {A12}\nB12 ={B12}')
    # # if s - 2 <= 0: print(f's - 2 = {s - 2}\nuxo = {dx_odt}\nuzo = {dz_odt}\nuxd = {dx_ddt}\nuzd ={dz_ddt}')
    # return [dx_odt, dz_odt, dx_ddt, dz_ddt]
    return A11, B11, L, M, A12, B11


def velo(Xox, Xoz, Xdx, Xdz, pp, a, dimm):
    p = pp
    dim = dimm
    dx_odt, dz_odt, dx_ddt, dz_ddt = diff_eq_near(Xox, Xoz, Xdx, Xdz, p, a, dim)

    uud = np.sqrt(dx_ddt ** 2 + dz_ddt ** 2)
    uuo = np.sqrt(dx_odt ** 2 + dz_odt ** 2)

    return [uuo, uud]

In [2]:
sep_dis = (0, 0.005, 0.01, 0.1, 0.25, 0.8, 1.8, 2.5, 2.7, 2.9, 3, 3.3, 3.6, 3.8, 4, 5, 7, 11)
size_ratio = (0.125, 0.25, 0.5, 1, 2, 4, 8)
aa1 = 2.5
aa2 = (2.5 / 8, 2.5 / 4, 2.5 / 2, 2.5, 2.5 * 2, 2.5 * 4, 2.5 * 8)
rrho = 0.1
FFz = -0.005
FFz_ne = -0.005
FFx = 0
pp = aa1, rrho, FFz, FFx
p_ne = aa1, rrho, FFz_ne, FFx
bb1 = (0.9942, 0.9729, 0.9272, 0.8905, 0.7642, 0.4734, 0.2377)
bb2 = (1.536, 3.843, 5.611, 5.772, 5.020, 3.710, 2.704)
bb3 = (-1.544, 0.3421, 4.404, 7.070, 5.604, 1.894, -0.840)
dd1 = (0.99968, 0.9951, 0.9537, 0.7750, 0.4768, 0.2488, 0.1250)
dd2 = (-0.00018, 0.0088, 0.1514, 0.9306, 2.277, 3.610, 5.620)
dd3 = (-0.003, -0.026, -0.194, -0.900, -2.188, -4.061, -8.500)
dd4 = (0, 0, -0.3, -2.0, -4.5, -6.4, -9.2)
ee1 = (1.518, 3.795, 5.600, 6.043, 5.600, 3.795, 1.518)
ee2 = (-1.536, 0.323, 4.179, 6.327, 4.179, 0.323, -1.536)
ll1 = (0.3205, 0.6040, 1.173, 2.000, 2.788, 3.759, 5.658)
ll2 = (-0.458, -0.679, -1.115, -1.800, -2.649, -4.224, -8.557)
ll3 = (0.356, -0.320, -1.63, -4.00, -5.17, -6.48, -9.16)
mm0 = (0.0108, 0.0596, 0.2142, 0.4021, 0.4077, 0.2451, 0.1148)
mm1 = (0.721, 1.391, 2.274, 2.967, 3.352, 3.097, 2.602)
mm2 = (-0.396, 0.440, 2.750, 5.088, 4.777, 1.918, -0.696)
Xox = 63.5
Xoz = 59.5
# Xdx = 63.5
# Xdz = 99.5

s_threshold = [3, 2.8, 2.5, 2.3, 2.1, 2.015]
s_1 = np.arange(2.000001,6,0.000001)
# legends = ['obstacle_x', 'obstacle_z', 'microrobto_x', 'microrobot_z']
# legends_1 = ['obstacle_far_x', 'obstacle_inte_x', 'obstacle_near_x']
# legends_total_vel = ['obstacle_far', 'obstacle_inte', 'obstacle_near',
#                      'microrobot_far', 'microrobot_inte', 'microrobot_near']
# legends_total_vel_x = ['obstacle_far_x', 'obstacle_inte_x', 'obstacle_near_x',
#                      'microrobot_far_x', 'microrobot_inte_x', 'microrobot_near_x']
legends = ['A11', 'A12', 'L', 'M', 'A12', 'B12']
# U = -FFz / (6 * np.pi * rrho * aa1)
# U = 1
# plt.style.use('dark_background')
s_up = 3
s_low = 3
i = 3
for s_cri in [3]: #s_threshold:
    s_indices_mask_near = np.where(s_1 <= s_low)
    s_indices_mask_far = np.where(s_1 > s_up)
    # s_indices_mask_inte = np.where((s < s_up) & (s > s_low))
    # r_1_near = (aa1 + aa2[3]) * s[s_indices_mask_near] / 2
    # r_1_far = (aa1 + aa2[3]) * s[s_indices_mask_far] / 2
    # # r_1_int = (aa1 + aa2[3]) * s[s_indices_mask_inte] / 2
    # # print(r_1)
    # theta = -30
    # xx_near = r_1_near * np.sin(theta)
    # zz_near = r_1_near * np.cos(theta)
    # xx_far = r_1_far * np.sin(theta)
    # zz_far = r_1_far * np.cos(theta)
    # xx_inte = r_1_int * np.sin(theta)
    # zz_inte = r_1_int * np.cos(theta)
    # print(f'xx = {xx}'
    #       f'zz = {zz}')

    dimm = bb1[i], bb2[i], bb3[i], dd1[i], dd2[i], dd3[i], dd4[i], ee1[i], ee2[i], ll1[i], ll2[i], ll3[i], mm0[i], mm1[
        i], mm2[i]
    # uox_f, uoz_f, udx_f, udz_f = diff_eq(xx,zz,pp,aa2[i], dimm)
    # uox_n, uoz_n, udx_n, udz_n = diff_eq_near(xx,zz,pp,aa2[i], dimm)
    m_f = s_far(s_1[s_indices_mask_far],aa1, aa2[i])
    # print(m_f
    m_n = s_near(s_1[s_indices_mask_near],aa1, aa2[i], dimm)
    # print(m_n)


In [ ]:
    # print(m_n)
    # u_int = ((s[s_indices_mask_inte] - s_low) * diff_eq(xx_inte, zz_inte, pp, aa2[i], dimm) + (
                # s_up - s[s_indices_mask_inte]) * diff_eq_near(xx_inte, zz_inte, pp, aa2[i], dimm)) / (s_up - s_low)

    # Define the interpolation function for u_inte using CubicSpline
    # s_inte = s[s_indices_mask_inte]
    # s_low = s_inte.min()
    # s_up = s_inte.max()
    # u_n_inte = diff_eq_near(xx_inte, zz_inte, pp, aa2[i], dimm)
    # u_f_inte = diff_eq(xx_inte, zz_inte, pp, aa2[i], dimm)
    # u_int = CubicSpline([s_low, s_up], [u_n_inte, u_f_inte])(s_inte)

    # Interpolate u_inte values for the intermediate s values
    # u_int = u_int_interp(s_inte)


    # u_f_t_o = np.sqrt((u_f[0]) ** 2 + (u_f[1]) ** 2)
    # u_n_t_o = np.sqrt((u_n[0]) ** 2 + (u_n[1]) ** 2)
    # u_i_t_o = np.sqrt((u_int[0]) ** 2 + (u_int[1]) ** 2)
    # u_f_t_m = np.sqrt((u_f[2]) ** 2 + (u_f[3]) ** 2)
    # u_n_t_m = np.sqrt((u_n[2]) ** 2 + (u_n[3]) ** 2)
    # u_i_t_m = np.sqrt((u_int[2]) ** 2 + (u_int[3]) ** 2)
    # print(u_i_t_m)
    plt.figure(figsize=(9, 7))
    colormap = plt.cm.get_cmap('RdYlBu', len(legends))
    for j in range(4):
        plt.plot(s_1[s_indices_mask_far], m_f[j] , label=f'{legends[j]}', color=colormap(j))
        plt.plot(s_1[s_indices_mask_near], m_n[j] ,  linestyle='--',color=colormap(j))
    # j = 1
    # plt.plot(s_1, m_f[:,j] , label=legends[j], color=colormap(j))
    # plt.plot(s_1, m_n[:,j] ,  color=colormap(j))

        
        # plt.plot(s[s_indices_mask_far], u_f[j] / U, label=legends_total_vel_x[0], color=colormap(0))
        # plt.plot(s[s_indices_mask_inte], u_int[j] / U, label=legends_total_vel_x[1], linestyle=':', color=colormap(1))
        # plt.plot(s[s_indices_mask_near], u_n[j] / U, label=legends_total_vel_x[2],linestyle='--', color=colormap(2))
        # plt.plot(s[s_indices_mask_far], u_f[j+2] / U, label=legends_total_vel_x[3], color=colormap(5))
        # plt.plot(s[s_indices_mask_inte], u_int[j+2] / U, label=legends_total_vel_x[4], linestyle=':', color=colormap(4))
        # plt.plot(s[s_indices_mask_near], u_n[j+2] / U, label=legends_total_vel_x[5], linestyle='--', color=colormap(3))
    # plt.plot(s[s_indices_mask_far], u_f_t_m / U, label=legends_total_vel[3], color=colormap(0))
    # plt.plot(s[s_indices_mask_inte], u_i_t_m / U, label=legends_total_vel[4], linestyle=':', color=colormap(1))
    # plt.plot(s[s_indices_mask_near], u_n_t_m / U, label=legends_total_vel[5], linestyle='--', color=colormap(2))
    # plt.plot(s[s_indices_mask_far], u_f_t_o / U, label=legends_total_vel[0], color=colormap(3))
    # plt.plot(s[s_indices_mask_inte], u_i_t_o / U, label=legends_total_vel[1], linestyle=':', color=colormap(4))
    # plt.plot(s[s_indices_mask_near], u_n_t_o / U, label=legends_total_vel[2], linestyle='--', color=colormap(5))

    plt.xlabel('s')
    # plt.tick_params(axis='x', color='white')
    # plt.tick_params(axis='y', color='white')
    plt.ylabel('mob_functions')
    plt.title(f' s_switch_over = {s_low}')
    plt.legend(frameon=False)
    plt.tight_layout()
    plt.savefig('mob_fun_vs_s_cric_2.015.eps',format='eps',dpi=300)
    plt.show()